<a href="https://colab.research.google.com/github/MihneaGherghel/FashionMinst/blob/main/FashionMinst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix
from torch.utils.data import random_split

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
train_set = torchvision.datasets.FashionMNIST("./data", download=True, transform=
                                                transforms.Compose([transforms.ToTensor()]))
train_size = int(0.8 * len(train_set))
val_size = len(train_set) - train_size
train_set, val_set = random_split(train_set, [train_size, val_size])
test_set = torchvision.datasets.FashionMNIST("./data", download=True, train=False, transform=
                                               transforms.Compose([transforms.ToTensor()]))

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=100)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=100)

In [ ]:
class FashionCNN(nn.Module):
  def __init__(self):
      super(FashionCNN, self).__init__()

      self.sequential1=nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=128, kernel_size=3, padding=1),
        #nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
      )

      self.sequential2=nn.Sequential(
        nn.Conv2d(in_channels=128, out_channels=512, kernel_size=3, padding=1),
        #nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
      )

      self.sequential3=nn.Sequential(
        nn.Conv2d(in_channels=512, out_channels=2048, kernel_size=3, padding=1),
        #nn.BatchNorm2d(2048),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
      )

      self.sequential4=nn.Sequential(
        nn.Conv2d(in_channels=2048, out_channels=8192, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
      )

      self.sequential5=nn.Sequential(
        nn.Flatten(),
        nn.Linear(8192,1000),
        nn.Dropout(0.2),
        nn.Linear(1000,500),
      )
      self.out=nn.Linear(500,10)

  def forward(self, x):
      x = self.sequential1(x)
      x = self.sequential2(x)
      x = self.sequential3(x)
      x = self.sequential4(x)
      x= self.sequential5(x)
      x=self.out(x)
      return x

In [ ]:
model = FashionCNN()
model.to(device)
error = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
num_epochs = 25

In [ ]:
count=0
predictions_list = []
labels_list = []
loss_list = []
iteration_list = []
accuracy_list = []
best_model=0
best_accuracy=100000
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        train = Variable(images.view(100, 1, 28, 28))
        labels = Variable(labels)

        outputs = model(train)
        loss = error(outputs, labels)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        count += 1

    # Testing the model

        if not (count % 50):    # It's same as "if count % 50 == 0"
            total = 0
            correct = 0

            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)

                test = Variable(images.view(100, 1, 28, 28))

                outputs = model(test)

                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_list.append(predictions)
                correct += (predictions == labels).sum()

                total += len(labels)

            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            if(accuracy<best_accuracy):
              best_accuracy=accuracy
              best_model=model

        if not (count % 500):
            print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))

Iteration: 500, Loss: 0.36577510833740234, Accuracy: 86.89166259765625%
Iteration: 1000, Loss: 0.31450822949409485, Accuracy: 89.03333282470703%
Iteration: 1500, Loss: 0.3291130065917969, Accuracy: 89.14166259765625%
Iteration: 2000, Loss: 0.18008233606815338, Accuracy: 90.75833129882812%
Iteration: 2500, Loss: 0.20080791413784027, Accuracy: 91.29999542236328%
Iteration: 3000, Loss: 0.1037425547838211, Accuracy: 90.3499984741211%
Iteration: 3500, Loss: 0.09745321422815323, Accuracy: 90.66666412353516%
Iteration: 4000, Loss: 0.19769905507564545, Accuracy: 90.56666564941406%
Iteration: 4500, Loss: 0.13788123428821564, Accuracy: 90.11666870117188%
Iteration: 5000, Loss: 0.2568441331386566, Accuracy: 90.125%
Iteration: 5500, Loss: 0.03657910227775574, Accuracy: 90.53333282470703%
Iteration: 6000, Loss: 0.1278616189956665, Accuracy: 90.74166870117188%
Iteration: 6500, Loss: 0.1916695535182953, Accuracy: 90.26666259765625%
Iteration: 7000, Loss: 0.033521637320518494, Accuracy: 91.25%
Iterati

In [ ]:
for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    labels_list.append(labels)

    test = Variable(images.view(100, 1, 28, 28))

    outputs = best_model(test)

    predictions = torch.max(outputs, 1)[1].to(device)
    predictions_list.append(predictions)
    correct += (predictions == labels).sum()

    total += len(labels)

accuracy = correct * 100 / total;
print(accuracy)

tensor(91.6364, device='cuda:0')


Rezultate: Test - 92.05